In [1]:
#调用接口
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorlayer as tl
from TransferLearningWrapV1 import TransferLearningMethod

In [2]:
#全局变量
IDcol = 'jdpin'
label ='card_desire'
file_dir = './'
pipeline_model = 'data_precessed_pipeline_{}.pkl'.format('Mscale')  #数据预处理pipeline
encoder_model = file_dir + 'model2/stack_encoder_lab_{}.ckpt'.format('Mscale') #堆栈式编码器模型
encoder_cols = ['jdmall_user_f0002','jdmall_user_p0001','jdmall_user_p0006','jdmall_user_p0019','jdmall_user_p0020','jdmall_up_m0001','jdmall_up_m0009','jdmall_up_m0027','pay_pay_p0004179','catenum','catenum_3m','catemum_6m','catemum_9m','catenum_1y','amount','amount_3m','amount_6m','qtty','qtty_3m','qtty_6m','qtty_1y','qtty_std_9m','catenum_std_3m','catenum_std_6m','worldwide_tax_amount_3m','pop_qtty_3m','self_qtty_3m','fresh_qtty_3m','mother_baby_qtty_3m','appliance_qtty_3m','phone_qtty_3m','clothes_qtty_3m','sport_qtty_3m','computer_office_qtty_3m','food_qtty_3m','pop_nord_3m','self_nord_3m','computer_office_nord_3m','food_nord_3m','self_amount_3m','virtual_amount_3m','fresh_amount_3m','mother_baby_amount_3m','car_amount_3m','pet_amount_3m','decoration_amount_3m','appliance_amount_3m','phone_amount_3m','clothes_amount_3m','sport_amount_3m','furniture_amount_3m','computer_office_amount_3m','wine_amount_3m','book_amount_3m','digit_amount_3m','shoes_amount_3m','recharge_amount_3m','ticket_amount_3m','worldwide_tax_amount_6m','self_qtty_6m','fresh_qtty_6m','mother_baby_qtty_6m','car_qtty_6m','pet_qtty_6m','decoration_qtty_6m','appliance_qtty_6m','phone_qtty_6m','clothes_qtty_6m','sport_qtty_6m','computer_office_qtty_6m','book_qtty_6m','digit_qtty_6m','shoes_qtty_6m','food_qtty_6m','recharge_qtty_6m','ticket_qtty_6m','pop_nord_6m','self_nord_6m','fresh_nord_6m','mother_baby_nord_6m','decoration_nord_6m','computer_office_nord_6m','book_nord_6m','food_nord_6m','self_amount_6m','fresh_amount_6m','mother_baby_amount_6m','pet_amount_6m','appliance_amount_6m','phone_amount_6m','clothes_amount_6m','sport_amount_6m','furniture_amount_6m','computer_office_amount_6m','book_amount_6m','digit_amount_6m','shoes_amount_6m','recharge_amount_6m','ticket_amount_6m','worldwide_tax_amount_12m','self_qtty_12m','fresh_qtty_12m','car_qtty_12m','pet_qtty_12m','decoration_qtty_12m','appliance_qtty_12m','phone_qtty_12m','clothes_qtty_12m','sport_qtty_12m','furniture_qtty_12m','wine_qtty_12m','digit_qtty_12m','shoes_qtty_12m','food_qtty_12m','dress_qtty_12m','recharge_qtty_12m','ticket_qtty_12m','pop_nord_12m','self_nord_12m','virtual_nord_12m','fresh_nord_12m','mother_baby_nord_12m','decoration_nord_12m','computer_office_nord_12m','book_nord_12m','food_nord_12m','self_amount_12m','virtual_amount_12m','fresh_amount_12m','mother_baby_amount_12m','car_amount_12m','appliance_amount_12m','phone_amount_12m','clothes_amount_12m','sport_amount_12m','furniture_amount_12m','computer_office_amount_12m','wine_amount_12m','book_amount_12m','digit_amount_12m','shoes_amount_12m','recharge_amount_12m','ticket_amount_12m','game_amount_3m','prc50_100_qtty_3m','prc100_500_qtty_3m','prc50_amount_3m','prc50_100_amount_3m','prc100_500_amount_3m','prc500_1000_amount_3m','prc5000_amount_3m','game_amount_6m','prc100_500_qtty_6m','prc500_1000_qtty_6m','prc1000_5000_qtty_6m','prc50_amount_6m','prc50_100_amount_6m','prc500_1000_amount_6m','prc5000_amount_6m','prc50_100_qtty_12m','prc100_500_qtty_12m','prc500_1000_qtty_12m','prc1000_5000_qtty_12m','prc5000_qtty_12m','prc50_amount_12m','prc500_1000_amount_12m','prc5000_amount_12m','top1_qtty_nord_3m','top1_qtty_nsku_3m','top1_qtty_amount_3m','top1_qtty_max_ord_amt_3m','top1_qtty_min_ord_amt_3m','top1_amt_max_ord_amt_3m','top1_amt_min_ord_amt_3m','top1_qtty_nord_6m','top1_qtty_nsku_6m','top1_qtty_max_ord_amt_6m','top1_qtty_min_ord_amt_6m','top1_amt_nord_6m','top1_amt_qtty_6m','top1_amt_max_ord_amt_6m','top1_amt_min_ord_amt_6m','top1_qtty_nsku_12m','top1_qtty_amount_12m','top1_qtty_max_ord_amt_12m','top1_qtty_min_ord_amt_12m','top2_qtty_nord_12m','top2_qtty_sale_qtty_12m','top2_qtty_nsku_12m','top2_qtty_max_ord_amt_12m','top2_qtty_min_ord_amt_12m','top1_amt_max_ord_amt_12m','top1_amt_min_ord_amt_12m','top2_amt_nord_12m','top2_amt_qtty_12m','top2_amt_nsku_12m','top2_amt_max_ord_amt_12m','top2_amt_min_ord_amt_12m','reg_user_type_cd','ord_day_cnt_1m','ord_all_1m','cancel_ord_1m','hdfk_ord_1m','ivc_ord_1m','off_ord_1m','ord_23_05_1m','ord_06_10_1m','ord_11_17_1m','ord_18_22_1m','weekend_ord_1m','weekday_ord_1m','self_ord_1m','virtual_ord_1m','amt_all_1m','hdfk_amt_1m','zt_delv_amt_1m','ivc_amt_1m','discount_amt_1m','mob_amt_1m','night_amt_1m','amt_23_05_1m','amt_06_10_1m','amt_11_17_1m','amt_18_22_1m','weekend_amt_1m','self_amt_1m','virtual_amt_1m','reject_amt_1m','refund_amt_1m','max_ord_amt_1m','min_ord_amt_1m','avg_ord_amt_1m','app_ord_1m','ord_day_cnt_3m','ord_all_3m','cancel_ord_3m','hdfk_ord_3m','online_ord_3m','off_ord_3m','night_ord_3m','ord_23_05_3m','ord_06_10_3m','ord_11_17_3m','ord_18_22_3m','weekend_ord_3m','weekday_ord_3m','self_ord_3m','virtual_ord_3m','amt_all_3m','hdfk_amt_3m','zt_delv_amt_3m','discount_amt_3m','night_amt_3m','amt_23_05_3m','amt_06_10_3m','amt_18_22_3m','weekend_amt_3m','reject_amt_3m','refund_amt_3m','ord_0_500_3m','ord_500_1000_3m','ord_1000_3000_3m','ord_3000_3m','app_ord_3m','ord_day_cnt_6m','ord_all_6m','cancel_ord_6m','hdfk_ord_6m','off_ord_6m','mob_ord_6m','night_ord_6m','ord_23_05_6m','ord_06_10_6m','ord_11_17_6m','ord_18_22_6m','weekend_ord_6m','virtual_ord_6m','refund_ord_6m','hdfk_amt_6m','zt_delv_amt_6m','night_amt_6m','amt_23_05_6m','amt_06_10_6m','amt_18_22_6m','weekend_amt_6m','reject_amt_6m','refund_amt_6m','ord_500_1000_6m','ord_1000_3000_6m','ord_3000_6m','min_ord_amt_6m','complete_ord_12m','cancel_ord_12m','hdfk_ord_12m','zt_delv_ord_12m','ivc_ord_12m','night_ord_12m','ord_23_05_12m','ord_06_10_12m','weekend_ord_12m','virtual_ord_12m','refund_ord_12m','amt_all_12m','cancel_amt_12m','zt_delv_amt_12m','ivc_amt_12m','discount_amt_12m','night_amt_12m','amt_23_05_12m','amt_06_10_12m','amt_18_22_12m','weekend_amt_12m','reject_amt_12m','refund_amt_12m','ord_500_1000_12m','ord_1000_3000_12m','ord_3000_12m','min_ord_amt_12m','avg_ord_amt_12m','app_ord_12m','name_3m','name_6m','name_9m','name_1y','address_9m','address_1y','mobile_9m','mobile_1y','jrlab_up_fin_insu_f0041','jrlab_up_xjk_open','jrlab_up_fin_f0003','jrlab_up_fin_f0010','jrlab_up_pay_f0054','jrlab_up_pay_f0036','jrlab_up_pay_f0038','jrlab_up_pay_f0005','jrlab_up_pay_f0004','jrlab_up_pay_f0006','jrlab_up_pay_f0007','jrlab_up_pay_f0037','jrlab_up_common_mobile_province','jrlab_up_common_mobile_city','jrlab_up_common_mobile_operator','jdmall_ordr_f0117','jdmall_ordr_f0122','jdmall_jdmordr_f0003656','jdmall_jdmordr_f0003687','jdmall_jdmordr_f0003689','jdmall_jdmordr_f0002261','jdmall_ordr_f0112','jdmall_ordr_f0111','jdmall_ordr_f0001','jdmall_ordr_f0107','jdmall_ordr_f0108','jdmall_up_m0002','jrlab_up_common_city_level','jrlab_up_common_county','jrlab_up_mob_stability_class','jrlab_up_addr_stability_class','jrlab_up_marriage_class','jrlab_up_common_privince','jdmall_jdmall_f0003019','jrlab_up_pay_app_amt_6m','jrlab_up_pay_app_cnt_6m','jrlab_up_pay_app_day_avg_amt_6m','jrlab_up_pay_app_day_avg_cnt_6m','jrlab_up_pay_app_max_amt_6m','jrlab_up_pay_hour_11_17_amt_6m','jrlab_up_pay_hour_11_17_cnt_6m','jrlab_up_pay_hour_11_17_day_cnt_6m','jrlab_up_pay_hour_11_17_day_avg_amt_6m','jrlab_up_pay_hour_11_17_day_avg_cnt_6m','jrlab_up_pay_hour_11_17_max_amt_6m','jrlab_up_pay_app_amt_12m','jrlab_up_pay_app_cnt_12m','jrlab_up_pay_app_day_avg_amt_12m','jrlab_up_pay_app_day_avg_cnt_12m','jrlab_up_pay_app_max_amt_12m','jrlab_up_pay_hour_06_10_amt_12m','jrlab_up_pay_hour_06_10_cnt_12m','jrlab_up_pay_hour_06_10_day_avg_amt_12m','jrlab_up_pay_hour_06_10_day_avg_cnt_12m','jrlab_up_pay_hour_06_10_max_amt_12m','jrlab_up_pay_hour_11_17_amt_12m','jrlab_up_pay_hour_11_17_cnt_12m','jrlab_up_pay_hour_11_17_day_cnt_12m','jrlab_up_pay_hour_11_17_day_avg_amt_12m','jrlab_up_pay_hour_11_17_day_avg_cnt_12m','jrlab_up_pay_hour_11_17_max_amt_12m','jrlab_up_pay_hour_18_22_amt_12m','jrlab_up_pay_hour_18_22_cnt_12m','jrlab_up_pay_hour_18_22_day_cnt_12m','jrlab_up_pay_hour_18_22_day_avg_amt_12m','jrlab_up_pay_hour_18_22_day_avg_cnt_12m','jrlab_up_pay_hour_18_22_max_amt_12m','jrlab_up_career_value']
new_features = False
model_type = 'MediumParity'
# new_num = 
transfer_learning_model = file_dir+ 'model2/transfer_learning_lab_{}.ckpt'.format(label)

In [3]:
def get_test_data(data_name):
    from preprocessing import DataProcessing
    dp = DataProcessing(['pin']) 
    data = pd.read_csv('./data/{}.csv'.format(data_name))
    y = data.loc[:,data_name]
    cols=['jdmall_user_f0002','jdmall_user_p0001','jdmall_user_p0006','jdmall_user_p0019','jdmall_user_p0020','jdmall_up_m0001','jdmall_up_m0009','jdmall_up_m0027','pay_pay_p0004179','catenum','catenum_3m','catemum_6m','catemum_9m','catenum_1y','amount','amount_3m','amount_6m','qtty','qtty_3m','qtty_6m','qtty_1y','qtty_std_9m','catenum_std_3m','catenum_std_6m','worldwide_tax_amount_3m','pop_qtty_3m','self_qtty_3m','fresh_qtty_3m','mother_baby_qtty_3m','appliance_qtty_3m','phone_qtty_3m','clothes_qtty_3m','sport_qtty_3m','computer_office_qtty_3m','food_qtty_3m','pop_nord_3m','self_nord_3m','computer_office_nord_3m','food_nord_3m','self_amount_3m','virtual_amount_3m','fresh_amount_3m','mother_baby_amount_3m','car_amount_3m','pet_amount_3m','decoration_amount_3m','appliance_amount_3m','phone_amount_3m','clothes_amount_3m','sport_amount_3m','furniture_amount_3m','computer_office_amount_3m','wine_amount_3m','book_amount_3m','digit_amount_3m','shoes_amount_3m','recharge_amount_3m','ticket_amount_3m','worldwide_tax_amount_6m','self_qtty_6m','fresh_qtty_6m','mother_baby_qtty_6m','car_qtty_6m','pet_qtty_6m','decoration_qtty_6m','appliance_qtty_6m','phone_qtty_6m','clothes_qtty_6m','sport_qtty_6m','computer_office_qtty_6m','book_qtty_6m','digit_qtty_6m','shoes_qtty_6m','food_qtty_6m','recharge_qtty_6m','ticket_qtty_6m','pop_nord_6m','self_nord_6m','fresh_nord_6m','mother_baby_nord_6m','decoration_nord_6m','computer_office_nord_6m','book_nord_6m','food_nord_6m','self_amount_6m','fresh_amount_6m','mother_baby_amount_6m','pet_amount_6m','appliance_amount_6m','phone_amount_6m','clothes_amount_6m','sport_amount_6m','furniture_amount_6m','computer_office_amount_6m','book_amount_6m','digit_amount_6m','shoes_amount_6m','recharge_amount_6m','ticket_amount_6m','worldwide_tax_amount_12m','self_qtty_12m','fresh_qtty_12m','car_qtty_12m','pet_qtty_12m','decoration_qtty_12m','appliance_qtty_12m','phone_qtty_12m','clothes_qtty_12m','sport_qtty_12m','furniture_qtty_12m','wine_qtty_12m','digit_qtty_12m','shoes_qtty_12m','food_qtty_12m','dress_qtty_12m','recharge_qtty_12m','ticket_qtty_12m','pop_nord_12m','self_nord_12m','virtual_nord_12m','fresh_nord_12m','mother_baby_nord_12m','decoration_nord_12m','computer_office_nord_12m','book_nord_12m','food_nord_12m','self_amount_12m','virtual_amount_12m','fresh_amount_12m','mother_baby_amount_12m','car_amount_12m','appliance_amount_12m','phone_amount_12m','clothes_amount_12m','sport_amount_12m','furniture_amount_12m','computer_office_amount_12m','wine_amount_12m','book_amount_12m','digit_amount_12m','shoes_amount_12m','recharge_amount_12m','ticket_amount_12m','game_amount_3m','prc50_100_qtty_3m','prc100_500_qtty_3m','prc50_amount_3m','prc50_100_amount_3m','prc100_500_amount_3m','prc500_1000_amount_3m','prc5000_amount_3m','game_amount_6m','prc100_500_qtty_6m','prc500_1000_qtty_6m','prc1000_5000_qtty_6m','prc50_amount_6m','prc50_100_amount_6m','prc500_1000_amount_6m','prc5000_amount_6m','prc50_100_qtty_12m','prc100_500_qtty_12m','prc500_1000_qtty_12m','prc1000_5000_qtty_12m','prc5000_qtty_12m','prc50_amount_12m','prc500_1000_amount_12m','prc5000_amount_12m','top1_qtty_nord_3m','top1_qtty_nsku_3m','top1_qtty_amount_3m','top1_qtty_max_ord_amt_3m','top1_qtty_min_ord_amt_3m','top1_amt_max_ord_amt_3m','top1_amt_min_ord_amt_3m','top1_qtty_nord_6m','top1_qtty_nsku_6m','top1_qtty_max_ord_amt_6m','top1_qtty_min_ord_amt_6m','top1_amt_nord_6m','top1_amt_qtty_6m','top1_amt_max_ord_amt_6m','top1_amt_min_ord_amt_6m','top1_qtty_nsku_12m','top1_qtty_amount_12m','top1_qtty_max_ord_amt_12m','top1_qtty_min_ord_amt_12m','top2_qtty_nord_12m','top2_qtty_sale_qtty_12m','top2_qtty_nsku_12m','top2_qtty_max_ord_amt_12m','top2_qtty_min_ord_amt_12m','top1_amt_max_ord_amt_12m','top1_amt_min_ord_amt_12m','top2_amt_nord_12m','top2_amt_qtty_12m','top2_amt_nsku_12m','top2_amt_max_ord_amt_12m','top2_amt_min_ord_amt_12m','reg_user_type_cd','ord_day_cnt_1m','ord_all_1m','cancel_ord_1m','hdfk_ord_1m','ivc_ord_1m','off_ord_1m','ord_23_05_1m','ord_06_10_1m','ord_11_17_1m','ord_18_22_1m','weekend_ord_1m','weekday_ord_1m','self_ord_1m','virtual_ord_1m','amt_all_1m','hdfk_amt_1m','zt_delv_amt_1m','ivc_amt_1m','discount_amt_1m','mob_amt_1m','night_amt_1m','amt_23_05_1m','amt_06_10_1m','amt_11_17_1m','amt_18_22_1m','weekend_amt_1m','self_amt_1m','virtual_amt_1m','reject_amt_1m','refund_amt_1m','max_ord_amt_1m','min_ord_amt_1m','avg_ord_amt_1m','app_ord_1m','ord_day_cnt_3m','ord_all_3m','cancel_ord_3m','hdfk_ord_3m','online_ord_3m','off_ord_3m','night_ord_3m','ord_23_05_3m','ord_06_10_3m','ord_11_17_3m','ord_18_22_3m','weekend_ord_3m','weekday_ord_3m','self_ord_3m','virtual_ord_3m','amt_all_3m','hdfk_amt_3m','zt_delv_amt_3m','discount_amt_3m','night_amt_3m','amt_23_05_3m','amt_06_10_3m','amt_18_22_3m','weekend_amt_3m','reject_amt_3m','refund_amt_3m','ord_0_500_3m','ord_500_1000_3m','ord_1000_3000_3m','ord_3000_3m','app_ord_3m','ord_day_cnt_6m','ord_all_6m','cancel_ord_6m','hdfk_ord_6m','off_ord_6m','mob_ord_6m','night_ord_6m','ord_23_05_6m','ord_06_10_6m','ord_11_17_6m','ord_18_22_6m','weekend_ord_6m','virtual_ord_6m','refund_ord_6m','hdfk_amt_6m','zt_delv_amt_6m','night_amt_6m','amt_23_05_6m','amt_06_10_6m','amt_18_22_6m','weekend_amt_6m','reject_amt_6m','refund_amt_6m','ord_500_1000_6m','ord_1000_3000_6m','ord_3000_6m','min_ord_amt_6m','complete_ord_12m','cancel_ord_12m','hdfk_ord_12m','zt_delv_ord_12m','ivc_ord_12m','night_ord_12m','ord_23_05_12m','ord_06_10_12m','weekend_ord_12m','virtual_ord_12m','refund_ord_12m','amt_all_12m','cancel_amt_12m','zt_delv_amt_12m','ivc_amt_12m','discount_amt_12m','night_amt_12m','amt_23_05_12m','amt_06_10_12m','amt_18_22_12m','weekend_amt_12m','reject_amt_12m','refund_amt_12m','ord_500_1000_12m','ord_1000_3000_12m','ord_3000_12m','min_ord_amt_12m','avg_ord_amt_12m','app_ord_12m','name_3m','name_6m','name_9m','name_1y','address_9m','address_1y','mobile_9m','mobile_1y','jrlab_up_fin_insu_f0041','jrlab_up_xjk_open','jrlab_up_fin_f0003','jrlab_up_fin_f0010','jrlab_up_pay_f0054','jrlab_up_pay_f0036','jrlab_up_pay_f0038','jrlab_up_pay_f0005','jrlab_up_pay_f0004','jrlab_up_pay_f0006','jrlab_up_pay_f0007','jrlab_up_pay_f0037','jrlab_up_common_mobile_province','jrlab_up_common_mobile_city','jrlab_up_common_mobile_operator','jdmall_ordr_f0117','jdmall_ordr_f0122','jdmall_jdmordr_f0003656','jdmall_jdmordr_f0003687','jdmall_jdmordr_f0003689','jdmall_jdmordr_f0002261','jdmall_ordr_f0112','jdmall_ordr_f0111','jdmall_ordr_f0001','jdmall_ordr_f0107','jdmall_ordr_f0108','jdmall_up_m0002','jrlab_up_common_city_level','jrlab_up_common_county','jrlab_up_mob_stability_class','jrlab_up_addr_stability_class','jrlab_up_marriage_class','jrlab_up_common_privince','jdmall_jdmall_f0003019','jrlab_up_pay_app_amt_6m','jrlab_up_pay_app_cnt_6m','jrlab_up_pay_app_day_avg_amt_6m','jrlab_up_pay_app_day_avg_cnt_6m','jrlab_up_pay_app_max_amt_6m','jrlab_up_pay_hour_11_17_amt_6m','jrlab_up_pay_hour_11_17_cnt_6m','jrlab_up_pay_hour_11_17_day_cnt_6m','jrlab_up_pay_hour_11_17_day_avg_amt_6m','jrlab_up_pay_hour_11_17_day_avg_cnt_6m','jrlab_up_pay_hour_11_17_max_amt_6m','jrlab_up_pay_app_amt_12m','jrlab_up_pay_app_cnt_12m','jrlab_up_pay_app_day_avg_amt_12m','jrlab_up_pay_app_day_avg_cnt_12m','jrlab_up_pay_app_max_amt_12m','jrlab_up_pay_hour_06_10_amt_12m','jrlab_up_pay_hour_06_10_cnt_12m','jrlab_up_pay_hour_06_10_day_avg_amt_12m','jrlab_up_pay_hour_06_10_day_avg_cnt_12m','jrlab_up_pay_hour_06_10_max_amt_12m','jrlab_up_pay_hour_11_17_amt_12m','jrlab_up_pay_hour_11_17_cnt_12m','jrlab_up_pay_hour_11_17_day_cnt_12m','jrlab_up_pay_hour_11_17_day_avg_amt_12m','jrlab_up_pay_hour_11_17_day_avg_cnt_12m','jrlab_up_pay_hour_11_17_max_amt_12m','jrlab_up_pay_hour_18_22_amt_12m','jrlab_up_pay_hour_18_22_cnt_12m','jrlab_up_pay_hour_18_22_day_cnt_12m','jrlab_up_pay_hour_18_22_day_avg_amt_12m','jrlab_up_pay_hour_18_22_day_avg_cnt_12m','jrlab_up_pay_hour_18_22_max_amt_12m','jrlab_up_career_value']
    data_X = data.loc[:,cols]
    data_X.replace(['a','Z'],-1,inplace=True)
    values = {'jrlab_up_pay_f0036': '2000-01-01', 'jrlab_up_pay_f0037':'2000-01-01', 'jdmall_ordr_f0001':'2000-01-01', 'jdmall_ordr_f0002':'2000-01-01'}
    data_X.fillna(value=values,inplace=True)
    # 时间类变量
    time_vars = ['jrlab_up_pay_f0036','jrlab_up_pay_f0037','jdmall_ordr_f0001']
    #异常时间数据处理
    for var in time_vars:
        data_X.loc[:,var] = data_X.loc[:,var].str[:10]
    # data_time = data.loc[:,time_vars]
    time_vars = ['jrlab_up_pay_f0036','jrlab_up_pay_f0037','jdmall_ordr_f0001']
    data_X = dp.deal_with_time_variables(data_X,time_vars, end_date='2018-11-17')
    data_X = data_X.fillna(-1)
    data_precessed_pipeline = joblib.load('data_precessed_pipeline_Mscale .pkl')
    data_X_scale = data_precessed_pipeline.transform(data_X)
    data_scale = pd.DataFrame(data_X_scale)
    data_scale.columns = cols
    data_scale[data_name] = y.values
    return data_scale

In [4]:
data = get_test_data(label)

--> RUN TIME: <deal_with_time_variables> : 0.015638351440429688


In [5]:
data['test_col1'] = 1
data['test_col2'] = 2

In [6]:
# 实例化迁移学习方法库
tlm = TransferLearningMethod(data, None, 'card_desire', encoder_cols)

In [7]:
#加载模型
act = tf.nn.relu
x = tf.placeholder(tf.float32, shape=[None, 396], name='x')
network = tl.layers.InputLayer(x, name='input_layer')
network = tl.layers.DropoutLayer(network, keep=0.8, name='denoising1')
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, n_units=200, act=act, name= 'relu1')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, n_units=100, act=act, name= 'relu2')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
network = tl.layers.DenseLayer(network, n_units=50, act=act, name='relu3')
#重载网络
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, save_path = encoder_model)

[TL] InputLayer  input_layer: (?, 396)
[TL] DropoutLayer denoising1: keep: 0.800000 is_fix: False
[TL] DropoutLayer drop1: keep: 0.800000 is_fix: False
[TL] DenseLayer  relu1: 200 relu
[TL] DropoutLayer drop2: keep: 0.500000 is_fix: False
[TL] DenseLayer  relu2: 100 relu
[TL] DropoutLayer drop3: keep: 0.500000 is_fix: False
[TL] DenseLayer  relu3: 50 relu
INFO:tensorflow:Restoring parameters from ./model2/stack_encoder_lab_Mscale.ckpt


In [ ]:
X = data.loc[:,encoder_cols].values

In [ ]:
X.shape

In [8]:
layer_out = tlm.get_layer_outputs(network, sess, x, layer_name='output')

[TL] DenseLayer  output: 1 No Activation
[TL]   [*] geting layers with output
[TL]   got   0: output/bias_add:0   (?, 1)


In [8]:
dev_pred = tlm.retrain_classifier_nets(network, sess, x )

匹配数据量： 2949
所传数据包含2个编码器外新特征...
--> 迁移学习类型：异构型迁移学习
[TL] DenseLayer  output: 1 No Activation
Epoch 1 of 100 took 0.168949s
   train loss: 1.294516
   val loss: 1.695792
Epoch 10 of 100 took 0.079895s
   train loss: 0.598021
   val loss: 0.597826
Epoch 20 of 100 took 0.080642s
   train loss: 0.555636
   val loss: 0.517679
Epoch 30 of 100 took 0.079394s
   train loss: 0.496538
   val loss: 0.484515
Epoch 40 of 100 took 0.079707s
   train loss: 0.394006
   val loss: 0.409839
Epoch 50 of 100 took 0.078919s
   train loss: 0.373364
   val loss: 0.383667
Epoch 60 of 100 took 0.082660s
   train loss: 0.364104
   val loss: 0.386202
Epoch 70 of 100 took 0.079977s
   train loss: 0.344017
   val loss: 0.352157
Epoch 80 of 100 took 0.081363s
   train loss: 0.403222
   val loss: 0.431726
Epoch 90 of 100 took 0.080626s
   train loss: 0.376754
   val loss: 0.405016
Epoch 100 of 100 took 0.079565s
   train loss: 0.368725
   val loss: 0.394677
Evaluation
   test loss: 0.380962
Model saved in file: ./model